# Preprocessing .mat for NWB

In [1]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import convert_to_nwb_for_AO
import scipy.io
from tqdm import tqdm
import platform
from IPython.display import clear_output
from contextlib import redirect_stdout


# Load data 

## .mat files

In [ ]:
WRAO039 = "data/mouse_anastasia/WR/AO039_20190626.mat"  
WRAO027 = "data/mouse_anastasia/WR/AO027_20181101.mat"
WnonRAO084 = "data/mouse_anastasia/WnonR/AO084_20210507.mat"
WnonRAO086 = "data/mouse_anastasia/WnonR/AO086_20210617.mat"

### Rewarded

In [ ]:

with h5py.File(WRAO039, 'r') as f:
    data_group = f['Data']
    data_ref = f["#refs#"]
    #print("Références trouvées :")
    #for key in data_ref.keys():
    #    print(f" - {key} → shape: {data_ref[key].shape}, dtype: {data_ref[key].dtype}")
    print("Contenu de 'Data' :")
    for key in data_group.keys():
        print(f" - {key} → shape: {data_group[key].shape}, dtype: {data_group[key].dtype}")


### Non-rewarded

In [ ]:

with h5py.File(WnonRAO086, 'r') as f:
    data_group = f['Data']
    data_ref = f["#refs#"]
    #print("Références trouvées :")
    #for key in data_ref.keys():
    #    print(f" - {key} → shape: {data_ref[key].shape}, dtype: {data_ref[key].dtype}")
    print("Contenu de 'Data' :")
    for key in data_group.keys():
        print(f" - {key} → shape: {data_group[key].shape}, dtype: {data_group[key].dtype}")

### Comparison


In [ ]:
with h5py.File(WRAO039, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WRAO039 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WRAO027, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WRAO027 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WnonRAO084, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WnonRAO084 = {key: data_group[key][()] for key in data_group.keys()}

with h5py.File(WnonRAO086, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	WnonRAO086 = {key: data_group[key][()] for key in data_group.keys()}

In [ ]:
# Comparaison WRAO039 et WRAO027
keys_WRAO039 = set(WRAO039.keys())
keys_WRAO027 = set(WRAO027.keys())

same_keys = keys_WRAO039 == keys_WRAO027
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WRAO039 :", keys_WRAO039 - keys_WRAO027)
    print("Clés uniquement dans WRAO027 :", keys_WRAO027 - keys_WRAO039)

In [ ]:
# Comparaison WnonRAO084 et WnonRAO086
keys_WnonRAO084 = set(WnonRAO084.keys())
keys_WnonRAO086 = set(WnonRAO086.keys())

same_keys = keys_WnonRAO084 == keys_WnonRAO086
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WnonRAO084 :", keys_WnonRAO084 - keys_WnonRAO086)
    print("Clés uniquement dans WnonRAO086 :", keys_WnonRAO086 - keys_WnonRAO084)

In [ ]:
# Comaraison WR et WnonR
keys_WR = set(WRAO039.keys())
keys_WnonR = set(WnonRAO084.keys())
same_keys = keys_WR == keys_WnonR
print("Les deux dictionnaires ont les mêmes clés :", same_keys)

if not same_keys:
    print("Clés uniquement dans WR :", keys_WR - keys_WnonR)
    print("Clés uniquement dans WnonR :", keys_WnonR - keys_WR)

    identical_keys = keys_WR & keys_WnonR
    print("Clés présentes dans les deux dictionnaires :", identical_keys)


## Try anything .mat

In [ ]:
WRAO039 = "data/mouse_anastasia/WR/AO039_20190626.mat"  
WRAO027 = "data/mouse_anastasia/WR/AO027_20181101.mat"
WnonRAO084 = "data/mouse_anastasia/WnonR/AO084_20210507.mat"
WnonRAO086 = "data/mouse_anastasia/WnonR/AO086_20210617.mat"

with h5py.File(WRAO027, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	data = {key: data_group[key][()] for key in data_group.keys()}
	
	

In [ ]:
with h5py.File(WRAO027, 'r') as f:
    data_group = f['Data'] if 'Data' in f else f
    data = {key: data_group[key][()] for key in data_group.keys()}
    
print(data["Area"].shape)
print(data["Area"])


In [ ]:
with h5py.File(WRAO027, 'r') as f:
    data_group = f['Data'] if 'Data' in f else f
    lfp_refs = data_group['LFPs'][()]  # shape (3,1)
    
    # Accès aux vrais tableaux
    lfp_data = [f[ref[0]][()] for ref in lfp_refs]
    
    for i, lfp in enumerate(lfp_data):
        print(f"LFP {i} shape:", lfp.shape)



# CSV file


In [ ]:

csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns

In [ ]:

mouse = ''.join(chr(c) for c in WnonRAO084['mouse'].flatten())
date = ''.join(chr(c) for c in WnonRAO084['date'].flatten())
session_name = f"{mouse}_{date}"  

try:
    subject_info = csv_data[csv_data['Session'].astype(str).str.strip() == session_name].iloc[0]
except IndexError:
    raise ValueError(f"Session {session_name} not found in the CSV file.")


 # TOTAL _-_-_-_-_-_-_-_
 

python convert_to_nwb_for_AO.py /Volumes/Petersen-Lab/z_LSENS/Share/Loris_Fabbro/AO/mat_files /Volumes/Petersen-Lab/z_LSENS/Share/Loris_Fabbro/AO/NWB_files

## Avec serveur

In [ ]:

input_folder  = "/Volumes/Petersen-Lab/z_LSENS/Share/Loris_Fabbro/AO/mat_files"
output_folder = "/Volumes/Petersen-Lab/z_LSENS/Share/Loris_Fabbro/AO/NWB_files"
importlib.reload(convert_to_nwb_for_AO)
convert_to_nwb_for_AO.convert_data_to_nwb_an(input_folder=input_folder, output_folder=output_folder , print_progress=False)


## Sans serveur

In [6]:
output_folder_local = "data"
filename = "data/mouse_anastasia/WR/AO039_20190626.mat" # Chemin vers le fichier .mat
importlib.reload(convert_to_nwb_for_AO)
convert_to_nwb_for_AO.convert_data_to_nwb_an_mat(mat_file=filename, output_folder=output_folder_local)

**************************************************************************
 NWB conversion data/mouse_anastasia/WR/AO039_20190626.mat
 
📃 Creating config file for NWB conversion :
📑 Created NWB file :
ephys Whisker Rewarded (WR+) mouse: The mouse was trained to lick within 1 s following a whisker stimulus (go trials) but not in the absence of the stimulus (no-go trials). High-density extracellular recordings were performed from mPFC and wS1 using Neuronexus A1x32-Poly2-10 mm-50 s-177 probes
     o 📌 Add general metadata
         - Subject metadata
         - Session metadata
         - Device metadata
         - Extracellular electrophysiology metadata
     o ⏸️ Add intervall container
     o 🧠 Add units container
     o ⚙️ Add processing container
         - Behavior data
         - No ephys data for AN sessions
 
🔎 Validating NWB file before saving...
     o ✅ File is valid, no errors detected.
 
💾 Saving NWB file
     o 📂 NWB file saved at:
         - data/AO039_20190626_160524.nwb


# Accessibilité 

In [ ]:
import h5py
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np

# Ouvre le fichier NWB
nwb_file = "data/AO084_20210507_174652.nwb"
io = NWBHDF5IO(nwb_file, mode='r')
nwb = io.read()
stim_times = nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["StimFlags"].timestamps[:]
stim_data = nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["StimFlags"].data[:]


In [ ]:
print(stim_times.shape)
print(stim_data.shape)


In [ ]:
behavior_events = nwb.processing['behavior'].data_interfaces['BehavioralEvents']
print("Contenu de BehavioralEvents :", list(behavior_events.fields.keys()))

In [ ]:
# Accède au conteneur interne
event_series = nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series

# Affiche les noms des TimeSeries comportementales
print("Séries dans BehavioralEvents :", list(event_series.keys()))

In [ ]:
nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["TrialOnsets"].timestamps[:]  # Accède aux timestamps des événements "TrialOnsets"
print(nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["StimFlags"].data[:])

In [ ]:
stim_times = nwb.processing['behavior'].data_interfaces['BehavioralEvents'].time_series["StimFlags"].timestamps[:]

In [ ]:
nwb_file

# Autres